In [3]:
import requests
from datetime import datetime, timedelta
import ccxt
import pandas as pd
import numpy as np
import talib
import settings
import time
import joblib
from logging import getLogger,Formatter,StreamHandler,FileHandler,INFO

# モデルの読み込み
model_y_buy = joblib.load('./model_y_buy.xz')
model_y_sell = joblib.load('./model_y_sell.xz')

# ccxtのパラメータ
symbol = 'BTC/JPY'	  # 購入予定のシンボル
product_code = 'FX_BTC_JPY'
bitflyer = ccxt.bitflyer()		 # 使用する取引所を記入
bitflyer.apiKey = settings.apiKey
bitflyer.secret = settings.secret


In [8]:
position = bitflyer.private_get_getpositions( params = { "product_code" : "FX_BTC_JPY" })
orders = bitflyer.fetch_open_orders(
	symbol = "FX_BTC_JPY",
	params = { "product_code" : "FX_BTC_JPY" })
print(position)
print(orders)

[]
[{'id': 'JRF20211231-040717-137786', 'clientOrderId': None, 'info': {'id': '0', 'child_order_id': 'JFX20211231-040717-173690F', 'product_code': 'FX_BTC_JPY', 'side': 'BUY', 'child_order_type': 'LIMIT', 'price': '5582151.0', 'average_price': '0.0', 'size': '0.01', 'child_order_state': 'ACTIVE', 'expire_date': '2022-01-30T04:07:17', 'child_order_date': '2021-12-31T04:07:17', 'child_order_acceptance_id': 'JRF20211231-040717-137786', 'outstanding_size': '0.01', 'cancel_size': '0.0', 'executed_size': '0.0', 'total_commission': '0.0'}, 'timestamp': 1640923637000, 'datetime': '2021-12-31T04:07:17.000Z', 'lastTradeTimestamp': None, 'status': 'open', 'symbol': 'FX_BTC_JPY', 'type': 'limit', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': 5582151.0, 'stopPrice': None, 'cost': 0.0, 'amount': 0.01, 'filled': 0.0, 'remaining': 0.01, 'fee': {'cost': 0.0, 'currency': None, 'rate': None}, 'average': None, 'trades': [], 'fees': [{'cost': 0.0, 'currency': None, 'rate': None}]}]


In [6]:
!pip3 install -U git+https://github.com/MtkN1/gmocoiner.git

  Cloning https://github.com/MtkN1/gmocoiner.git to c:\users\cheap\appdata\local\temp\pip-req-build-n1epkz82
  Resolved https://github.com/MtkN1/gmocoiner.git to commit 777bb8c5f2d26c67fb05aade3a5871be1844f871
  Created wheel for gmocoiner: filename=gmocoiner-0.1-py3-none-any.whl size=4501 sha256=e9d0daca606259d4c3f4dff4430b61501c3d01182fbcbffdcc93d7290c479a8e
  Stored in directory: C:\Users\cheap\AppData\Local\Temp\pip-ephem-wheel-cache-5k47lpd5\wheels\0a\48\2b\a5869a37876962a18061ccc2feeccf000a72ecb43c12219d02
Successfully built gmocoiner


  Running command git clone -q https://github.com/MtkN1/gmocoiner.git 'C:\Users\cheap\AppData\Local\Temp\pip-req-build-n1epkz82'


In [12]:
import asyncio
from gmocoiner import GMOCoin
import urllib.parse
import urllib.request
import json
import pandas as pd

def get_api(symbol: str) -> str:
	endPoint = "https://api.coin.z.com/public"
	klines = "/v1/klines"
	#params = {"symbol": symbol, "interval": "1day", "date": year}
	params = {"symbol": symbol, "interval": "15min", "date": "20220110"}
	url = endPoint + klines + "?" + urllib.parse.urlencode(params)
	req = urllib.request.Request(url)
	with urllib.request.urlopen(req) as response:
		return response.read()

def json_to_df(json_str: str) -> pd.DataFrame:
    d = json.loads(json_str)
    df = pd.json_normalize(d, record_path="data")
    df = df.astype("float64")
    df["openTime"] = pd.to_datetime(df["openTime"], unit="ms")
    df.set_index("openTime", inplace=True)
    df.columns = ["Open", "High", "Low", "Close", "Volume"]
    return df.tz_localize("UTC").tz_convert("Asia/Tokyo")


print(get_api('BCH_JPY'))

b'{"status":0,"data":[{"openTime":"1641762000000","open":"43885","high":"43885","low":"43885","close":"43885","volume":"3"},{"openTime":"1641764700000","open":"43905","high":"43905","low":"43905","close":"43905","volume":"3"},{"openTime":"1641769200000","open":"43905","high":"43905","low":"43635","close":"43635","volume":"10"},{"openTime":"1641770100000","open":"43570","high":"43570","low":"43493","close":"43493","volume":"10.8"},{"openTime":"1641771000000","open":"43510","high":"43510","low":"43510","close":"43510","volume":"5.2"},{"openTime":"1641771900000","open":"43613","high":"43613","low":"43613","close":"43613","volume":"0.2"},{"openTime":"1641772800000","open":"43540","high":"43669","low":"43395","close":"43607","volume":"9.5"},{"openTime":"1641773700000","open":"43617","high":"43617","low":"43617","close":"43617","volume":"0.2"},{"openTime":"1641775500000","open":"43291","high":"43369","low":"43291","close":"43369","volume":"16"},{"openTime":"1641776400000","open":"43370","hig

In [3]:
import requests
import json
import hmac
import hashlib
import time
from datetime import datetime

apiKey    = 'yirXnei+HGvRm9G9k9gC18OOQB2cUwqR'
secretKey = '1fSPh31TeYXQ36jTCetCngHN7gZUuPNqRxPts4dyobBpxZZFNu+GmheTAWlP3DxH'

timestamp = '{0}000'.format(int(time.mktime(datetime.now().timetuple())))
method    = 'POST'
endPoint  = 'https://api.coin.z.com/private'
path      = '/v1/cancelBulkOrder'
reqBody = {
    "symbols": ["BTC","BTC_JPY"],
    "side": "BUY",
    "settleType": "OPEN",
    "desc": True
}

text = timestamp + method + path + json.dumps(reqBody)
sign = hmac.new(bytes(secretKey.encode('ascii')), bytes(text.encode('ascii')), hashlib.sha256).hexdigest()

headers = {
    "API-KEY": apiKey,
    "API-TIMESTAMP": timestamp,
    "API-SIGN": sign
}

res = requests.post(endPoint + path, headers=headers, data=json.dumps(reqBody))
print (json.dumps(res.json(), indent=2))

{
  "status": 0,
  "data": [],
  "responsetime": "2022-01-10T05:12:46.208Z"
}
